In [1]:
import numpy as np 
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.layers import LSTM
seed = 42
np.random.seed(seed)
import collections

/Users/thakur/newenv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_dict = dict()

global train_dict

def data_process(file_name, train=True):
    file = open(file_name,'r')
    if train:
        label = []
        no_of_data = [] # for creating the dictionary, gather all data
        train_x = [] #
        for line in file:
            lx = line.split('\t')
            ly = lx[1].rstrip().split(',')
            label.append(lx[0]) # labels appending
            train_x.append(ly) # used for numbering array
            no_of_data.extend(ly) # used for counting unique numbers
        
        #for calculating the line length
        max_len = max([len(i) for i in train_x])
        min_len = min([len(i) for i in train_x])
        mean_len = np.mean([len(i) for i in train_x])
        std_len = np.std([len(i) for i in train_x])

        print('max sentence length: ', max_len)
        print('min sentence length: ', min_len)
        print('mean:', mean_len)
        print('std:', std_len)

        # creating label array of shape(number,) 1-D row
        train_label = np.asarray(label).reshape(-1,)
        print('label array shape:', train_label.shape)
        
        # for checking the frequency
        di = collections.Counter(no_of_data)
        print('di', di)
        
        
        ddd = dict(di)
        dddd = sorted([[value,key] for (key,value) in ddd.items()], reverse=True)

        for i in range(len(dddd)):
            dddd[i][0] = i+2

        d = dict(dddd)

        res = {v:k for k,v in d.items()}
        
        # so we create if the number is greater than 99 as '1'. So our vocabulary has '1' to '99'.
        # by adding '0', vocabulary has '0' to '99'
        for i,j in res.items():
            if res[i]>99:
                res[i] = 1
        
        
        
        # here you can create unique keys
        for i,j in enumerate(no_of_data):
            train_dict[j] = i

        print('unique words:',len(train_dict.keys()))

        # making the dictionary start with (one) zero (+1, = start with '1') not in the or
        #for i,j in enumerate(train_dict.keys()):
        #    train_dict[j] = i+1

        file.close()
        
        
        
        
        train_data = np.asarray(train_x).reshape(-1,)
        for index,i in enumerate(train_data):
            for pos,k in enumerate(i):
                train_data[index][pos] = res[train_data[index][pos]]#train_dict[train_data[index][pos]]

        return (train_data, train_label, di)
    
    else:
        
        test_x = [] #
        for line in file:
            #lx = line.split('\t')
            ly = line.rstrip().split(',')
            test_x.append(ly) # used for labeling
            
        max_len = max([len(i) for i in test_x])
        min_len = min([len(i) for i in test_x])
        mean_len = np.mean([len(i) for i in test_x])
        std_len = np.std([len(i) for i in test_x])

        print('max sentence length: ',max_len)
        print('min sentence length: ', min_len)
        print('mean:', mean_len)
        print('std:', std_len)

        file.close()
        test_data = np.asarray(test_x).reshape(-1,)
        for index,i in enumerate(test_data):
            for pos,k in enumerate(i):
                test_data[index][pos] = train_dict[test_data[index][pos]]
                
        return test_data

In [3]:
data, target, dii = data_process('training-data-small.txt')

max sentence length:  100
min sentence length:  2
mean: 26.2512
std: 17.84020455488109
label array shape: (10000,)
di Counter({'Z4': 9642, 'Z15': 8314, 'X344420902': 6718, 'Y3143': 4832, 'Y1222': 4451, 'Y1940': 4351, 'Y2817': 4342, 'Y1555': 4193, 'Y2321': 4081, 'Y1522': 3890, 'X123474229': 3798, 'Y2307': 3698, 'Y1478': 3429, 'Y1080': 3379, 'Y2170': 3343, 'Y1132': 3326, 'Y3269': 3301, 'Y1702': 3296, 'X773579': 3234, 'Y1486': 3196, 'Y1705': 3105, 'Y2949': 3095, 'Y2680': 3004, 'Y1725': 2965, 'Y2793': 2954, 'Y3232': 2887, 'Y3021': 2876, 'Y2814': 2758, 'Y2072': 2723, 'Y3145': 2697, 'Y2667': 2663, 'X127436348': 2638, 'Y2640': 2636, 'Y1690': 2633, 'Y2024': 2632, 'Y1468': 2630, 'Y1791': 2609, 'Y3346': 2605, 'Y1353': 2602, 'Y3585': 2566, 'Y1238': 2559, 'Y2267': 2526, 'Y1456': 2506, 'Y977': 2506, 'Y2': 2479, 'Y2257': 2473, 'Y3250': 2466, 'Y2684': 2464, 'X62716661': 2464, 'Y1367': 2431, 'Y2849': 2421, 'Y2641': 2411, 'Y3233': 2409, 'Y1206': 2401, 'Y3141': 2387, 'Y2794': 2382, 'X51769735': 2368, 'X

In [4]:
print(data)

[list([20, 34, 30, 2, 3])
 list([77, 12, 74, 86, 69, 59, 33, 84, 4, 65, 91, 58, 66, 64, 85, 81, 98, 71, 13, 18, 10, 5, 8, 43, 24, 47, 57, 26, 28, 27, 2, 3])
 list([83, 20, 4, 7, 22, 2, 3, 1]) ...
 list([4, 18, 17, 29, 43, 47, 57, 26, 48, 54, 27, 15, 9, 6, 30, 42, 31, 56, 5, 38, 19, 24, 22, 49, 32, 25, 53, 34, 16, 55, 41, 35, 51, 40, 23, 28, 60, 13, 52, 14, 10, 21, 36, 11, 8, 39, 45, 7, 44, 37, 2, 3, 96])
 list([1, 70, 4, 72, 57, 52, 2, 3, 1])
 list([12, 86, 61, 80, 62, 59, 58, 1, 1, 74, 4, 72, 83, 1, 82, 46, 17, 29, 54, 15, 9, 6, 42, 31, 5, 19, 24, 55, 51, 40, 11, 8, 7, 2, 3])]


In [5]:
print(target)

['0' '0' '1' ... '1' '1' '0']


ddd = dict(dii)
dddd = sorted([[value,key] for (key,value) in ddd.items()], reverse=True)
        
for i in range(len(dddd)):
    dddd[i][0] = i+2
        
d = dict(dddd)
        
res = {v:k for k,v in d.items()}
        
for i,j in res.items():
    if res[i]>99:
        res[i] = 1

In [6]:
max_words = 50 # maximum sequence length
data = sequence.pad_sequences(data, maxlen=max_words)
#X_test = sequence.pad_sequences(X_test, maxlen=max_words)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

In [8]:
print(X_train.shape)

(8000, 50)


In [ ]:
# create the model
top_words = 100
max_words = 50
model = Sequential()
model.add(Embedding(top_words, 64, input_length=max_words))
#model.add(Flatten())
model.add(LSTM(128))
model.add(Dropout(0.2))
#model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


In [ ]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32, verbose=2)
# Final evaluation of the model
#scores = model.evaluate(X_test, y_test, verbose=0)
#print("Accuracy: %.2f%%" % (scores[1]*100))